# Transfer Learning Using a Pre-Trained Model

```python
ImageDataGenerator(rescale=1./255, validation_split=0.2)

model.fit(...)
```

## Import libraries

In [ ]:
# !pip install tensorflow==2.16.2 matplotlib==3.9.1
# import numpy as np
# import matplotlib.pyplot as plt
# from tensorflow.keras.applications import VGG16
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Flatten
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

---

## 1 Load Pre-Trained Model

- `include_top=False`: 不加入顶层的fully connected layer 
- `layer.trainable=False`: 不trainable - 意思是，不要更新这些层的权重，**冻结这些层（冻结训练）**

In [ ]:
# Load Pre-Trained VGG16 Model on ImageNet
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

## 2 Create and compile the model

In [ ]:
# Create a new model and add the base model and new layers
model = Sequential([
    base_model,
    Flatten(),   # Flatten the output of the base model to a 1D vector
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')  # Change to the number of classes you have
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## 3 Create images

In [ ]:
import os
from PIL import Image
import numpy as np

# Create directories if they don't exist
os.makedirs('sample_data/Img2Classes/class_a', exist_ok=True)
os.makedirs('sample_data/Img2Classes/class_b', exist_ok=True)

# Create 10 sample images for each class
for i in range(10):
    # Create a blank white image for class_a
    img = Image.fromarray(np.ones((224, 224, 3), dtype=np.uint8) * 255)
    img.save(f'sample_data/Img2Classes/class_a/img_{i}.jpg')

    # Create a blank black image for class_b
    img = Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))
    img.save(f'sample_data/Img2Classes/class_b/img_{i}.jpg')

print("Sample images created in 'sample_data/'")

## 4 Train the model

In [ ]:
# Load and preprocess the dataset
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    'sample_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Verify if the generator has loaded images correctly
print(f"Found {train_generator.samples} images belonging to {train_generator.num_classes} classes.")

# Train the model
if train_generator.samples > 0:
    model.fit(train_generator, epochs=10)

## 5 Fine-Tune the model

Unfreeze the top layers of the base model and fine-tune.

In [ ]:
# Unfreeze the top layers of the base model 

for layer in base_model.layers[-4:]:
    layer.trainable = True 

# Compile the model again 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

# Train the model again 
model.fit(train_generator, epochs=10) 